In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
#Taiwan bounds: 0-50 N,80-160 E
#NEUSA bounds: 10-60 N,30-115 W
#These are large enough that an AR invading the NEUSA or Taiwan would be detected - i.e. >2000 km on all sides
#ds=xr.open_dataset(dir+'ls_extreme_rain_taiwan.nc')
#dates=ds.large_scale_extreme_days

dz2=xr.open_dataset(dir+'ivt_basics_neusa_90.nc')
longitude_sub=dz2.longitude.values
ds2=xr.open_dataset(dir+'ivt_basics_neusa.nc')#change this threshold for sensitivity testing
ds2['longitude']=longitude_sub
#ivt_threshold=528.6942100292907
ivt_threshold=ds2['ivt_threshold']
ivt_mag=ds2['ivt_mag']
ivtx=ds2['ivtx']
ivty=ds2['ivty']
lons_full=ivt_mag.longitude.values
lats_full=ivt_mag.latitude.values
print(ivt_mag)

lat_range=np.arange(0,60,0.25)
lon_range=np.arange(-140,-110,0.25)

if 1==1:
    ds3=xr.open_dataset(dir+'era_5_fall_u_850_neusa.nc')
    u_850=ds3['u']#.sel(latitude=lat_range,longitude=lon_range)
    ds4=xr.open_dataset(dir+'era_5_fall_v_850_neusa.nc')
    v_850=ds4['v']#.sel(latitude=lat_range,longitude=lon_range)
    ds5=xr.open_dataset(dir+'era_5_fall_mslp_neusa.nc')
    print(ds5)
    slp=ds5['msl']#.sel(latitude=lat_range,longitude=lon_range)/100.

    ds6=xr.open_dataset(dir+'era_5_fall_sfc_remnants_neusa.nc')
    ds6['longitude']=ds3.longitude.values
    print(ds6)
    slp_2=ds6['mslp']#.sel(latitude=lat_range,longitude=lon_range)
    ds7=xr.open_dataset(dir+'era_5_fall_pl_remnants_neusa.nc')
    ds7['longitude']=ds3.longitude.values
    u_850_2=ds7['u']#.sel(latitude=lat_range,longitude=lon_range)
    v_850_2=ds7['v']#.sel(latitude=lat_range,longitude=lon_range)

In [ ]:
ds=xr.open_dataset(dir+'extreme_days_ghcnd_99station_80area95.nc')
lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
print(dates)

ds1=xr.open_dataset(dir+'station_numbers_95.nc')
print(ds1)
lats1=ds1['lats']#use these lists for plotting
lons1=ds1['lons']+360.
stations1=ds1['stations']
thresholds=ds1['thresholds_annual_99']/10.

In [ ]:
#to draw grid lines on Lambert Conformal projection; 
#CREDIT ajdawson on GitHub https://gist.github.com/ajdawson/dd536f786741e987ae4e

from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
ds6=xr.open_dataset(dir+'IBTrACS.NA.v04r00.nc')
#print(ds)
lat_tc=ds6.lat
lon_tc=ds6.lon
#nature_tc=ds6.nature
ds7=xr.open_dataset(dir+'ibtracs_na_time.nc')
years_tc=ds7.years
months_tc=ds7.months
days_tc=ds7.days
hours_tc=ds7.hours
nature_tc=ds7.nature#1 if tropical, 0 if not
print(nature_tc)
print(hours_tc)

In [ ]:
dsa=xr.open_dataset(dir+'neusa_ar_climo_fall.nc')
ar=dsa.ar
tc_trop=dsa.tc_trop
tc_nontrop=dsa.tc_nontrop
labeled=dsa.labeled

In [ ]:
#time_step=50
#ivt_threshold=528.6942100292907
print(ivt_threshold)
import matplotlib as mpl
lats2=ds2.latitude.values.tolist()
lons2=ds2.longitude.values.tolist()
zipped=list(zip(dates,lats,lons,obs))
print(zipped[0:10])
zipped_thresh=list(zip(thresholds,lats1,lons1))
month_dict={'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}    
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
count_fig=0
count_file=0
dates_unique=[dt.datetime(1996,10,20,6)]
for d in range(len(dates_unique)):
    print(d)
    date=dates_unique[d]#date is at 6Z automatically.  IVT goes from 6Z on day to 6Z on next day
    select_date=[x for x in zipped if pd.to_datetime(x[0])==pd.to_datetime(date)]
    print(len(select_date))
    date_midday=date+dt.timedelta(hours=6)

    date_range=[date_midday]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT, AR, EP comparison
    
    labeled_four=labeled.sel(time=date_range)
    points_closest_list=[]
    obs_extreme_list=[]
    lats_extreme_list=[]
    lons_extreme_list=[]
    four_labeled_rain=xr.zeros_like(labeled_four)
    for i in range(len(lats1)):
        lat=lats1[i]
        lon=lons1[i]
        select_station=[x for x in select_date if x[1]==lat and x[2]==lon]
        select_thresh=[x for x in zipped_thresh if x[1]==lat and x[2]==lon]
        if len(select_station)>1:
            print('ERROR')
        if len(select_station)>0:
            obs_1=select_station[0][-1]
            thresh=select_thresh[0][0].values
            if obs_1>=thresh:
                obs_extreme_list.append(obs_1)
                lats_extreme_list.append(lat)
                lons_extreme_list.append(lon)
                
    ar_date_four=ar.sel(time=date_range)


    for date in date_range:
        print(date)
        date=pd.to_datetime(date)
        
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        
        ivt_mag_current=ivt_mag.sel(time=date)
        ivtx_current=ivtx.sel(time=date)
        ivty_current=ivty.sel(time=date)

        if month==12 and day==1:
            u_850_select=u_850_2.sel(time=date)
            v_850_select=v_850_2.sel(time=date)
            slp_select=slp_2.sel(time=date)
        else:
            u_850_select=u_850.sel(time=date)
            v_850_select=v_850.sel(time=date)
            slp_select=slp.sel(time=date)

        ivt_extreme=ivt_mag_current.where(ivt_mag_current>=ivt_threshold)
        print(ivt_extreme)
        ivt_250=ivt_mag_current.where(ivt_mag_current>=250.)
        ivt_ones=ivt_extreme/ivt_extreme
        ivt_ones=ivt_ones.fillna(0)
        
        ar_select=ar_date_four.sel(time=date)
        
        if 1==1:
            fig = plt.figure(figsize=(25,16))
            ax=plt.subplot(1,1,1,projection=proj_map)
        
        check_lat_tc=lat_tc.where(years_tc==year,drop=True)
        check_lat_tc=check_lat_tc.where(months_tc==month,drop=True)
        check_lat_tc=check_lat_tc.where(days_tc==day,drop=True)
        check_lat_tc=check_lat_tc.where(hours_tc==hour,drop=True)
        #TS,SS,EX,ET,
        check_lat_tc_trop=check_lat_tc.where(nature_tc==1,drop=True)
        check_lat_tc_nontrop=check_lat_tc.where(nature_tc!=1,drop=True)
        
        check_lat_tc_trop_list=[]
        for i in range(check_lat_tc_trop.storm.size):
            check_lat_tc_trop_values=check_lat_tc_trop[i,:].values.tolist()
            check_lat_tc_trop_values=[x for x in check_lat_tc_trop_values if x>0]
            if len(check_lat_tc_trop_values)>0:
                check_lat_tc_trop_value=check_lat_tc_trop_values[0]
                check_lat_tc_trop_list.append(check_lat_tc_trop_value)
                
        check_lat_tc_nontrop_list=[]
        for i in range(check_lat_tc_nontrop.storm.size):
            check_lat_tc_nontrop_values=check_lat_tc_nontrop[i,:].values.tolist()
            check_lat_tc_nontrop_values=[x for x in check_lat_tc_nontrop_values if x>0]
            if len(check_lat_tc_nontrop_values)>0:
                check_lat_tc_nontrop_value=check_lat_tc_nontrop_values[0]
                check_lat_tc_nontrop_list.append(check_lat_tc_nontrop_value)
        
        check_lon_tc=lon_tc.where(years_tc==year,drop=True)
        check_lon_tc=check_lon_tc.where(months_tc==month,drop=True)
        check_lon_tc=check_lon_tc.where(days_tc==day,drop=True)
        check_lon_tc=check_lon_tc.where(hours_tc==hour,drop=True)
        check_lon_tc_trop=check_lon_tc.where(nature_tc==1,drop=True)
        check_lon_tc_nontrop=check_lon_tc.where(nature_tc!=1,drop=True)
        
        check_lon_tc_trop_list=[]
        for i in range(check_lon_tc_trop.storm.size):
            check_lon_tc_trop_values=check_lon_tc_trop[i,:].values.tolist()
            check_lon_tc_trop_values=[x for x in check_lon_tc_trop_values if x>-1000]
            if len(check_lon_tc_trop_values)>0:
                check_lon_tc_trop_value=check_lon_tc_trop_values[0]
                check_lon_tc_trop_list.append(check_lon_tc_trop_value)
                
        check_lon_tc_nontrop_list=[]
        for i in range(check_lon_tc_nontrop.storm.size):
            check_lon_tc_nontrop_values=check_lon_tc_nontrop[i,:].values.tolist()
            check_lon_tc_nontrop_values=[x for x in check_lon_tc_nontrop_values if x>-1000]
            if len(check_lon_tc_nontrop_values)>0:
                check_lon_tc_nontrop_value=check_lon_tc_nontrop_values[0]
                check_lon_tc_nontrop_list.append(check_lon_tc_nontrop_value)
        
        if len(check_lat_tc_trop_list)>0 and len(check_lon_tc_trop_list)>0:
            for i in range(len(check_lat_tc_trop_list)):
                lat_tc_single=check_lat_tc_trop_list[i]
                lon_tc_single=check_lon_tc_trop_list[i]
                ax.plot(lon_tc_single,lat_tc_single,marker='*',color='r',transform=ccrs.PlateCarree(),markersize=24,zorder=41)
                
        if len(check_lat_tc_nontrop_list)>0 and len(check_lon_tc_nontrop_list)>0:
            for i in range(len(check_lat_tc_nontrop_list)):
                lat_tc_single=check_lat_tc_nontrop_list[i]
                lon_tc_single=check_lon_tc_nontrop_list[i]
                ax.plot(lon_tc_single,lat_tc_single,marker='*',color='saddlebrown',transform=ccrs.PlateCarree(),markersize=24,zorder=41)
                
        if 1==1:             
            ax.coastlines(resolution='10m')
            ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
            ax.add_feature(cfeature.LAKES.with_scale('50m'))
            countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
            ax.add_feature(countries)
            ax.set_extent([-100,-45,18,53],crs=ccrs.PlateCarree())

            # *must* call draw in order to get the axis boundary used to add ticks:
            fig.canvas.draw()

            # Define gridline locations and draw the lines using cartopy's built-in gridliner:
            xticks = [-135,-125,-115,-105,-95,-85,-75,-65,-55,-45,-35,-25]
            yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
            ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
            ax.tick_params(labelsize=26)
            # Label the end-points of the gridlines using the custom tick makers:
            ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
            ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
            lambert_xticks(ax, xticks)
            lambert_yticks(ax, yticks)

            cax=ax.scatter(lons_extreme_list,lats_extreme_list,s=240,c=obs_extreme_list,transform=ccrs.PlateCarree(),vmin=0,vmax=120,cmap=plt.cm.RdYlGn_r,zorder=10)
            cax1=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ivt_250,levels=np.arange(250,1040,50),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5)
            ax.set_title(str(hour)+'Z '+month_dict[str(date.month)]+' '+str(date.day)+', '+str(date.year),fontsize=46)
            cbar=plt.colorbar(cax1,pad=0,fraction=0.046)
            cbar.set_label('kg/m/s',fontsize=28,rotation=90,labelpad=10)
            cbar.ax.tick_params(labelsize=26)

            c=ax.contour(slp_select.longitude,slp_select.latitude,slp_select/100.,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))
            plt.clabel(c,fmt='%.00f',fontsize=22)
            X=6
            q=ax.quiver(u_850_select.longitude[0::X].values,u_850_select.latitude[0::X].values,u_850_select[0::X,0::X].values,v_850_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=30,width=0.04,alpha=0.5)
            ax.quiverkey(q,0.94,1.01,10,'10 m/s',fontproperties={'size':20})

            ax.set_title(str(hour)+'Z '+month_dict[str(date.month)]+' '+str(date.day)+', '+str(date.year),fontsize=42)
            cax21=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ivt_250.where(ar_select==1),levels=np.arange(250,1040,50),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5,label='Identified AR')
            cax2=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ar_select,[0,1],colors='b',transform=ccrs.PlateCarree(),hatches=[None,'.'],alpha=0.1,label='Identified AR',zorder=20)

            mpl.rcParams['hatch.linewidth']=0.01
            ax.text(-115,49.05,'Class: TC-linked AR',transform=ccrs.PlateCarree(),fontsize=28)
            plt.tight_layout()
            plt.show()

In [ ]:
fig.savefig(dir+'neusa_example_synoptic_10.png')

In [ ]:
if 1==1:#part 2: read the obs at eligible stations into a better format
    yr_start=1979
    yr_end=2019
    tuples=[]
    years=[]
    months=[]
    days1=[]
    lats1=[]
    lons1=[]
    elevs1=[]
    observations=[]
    dates=[]
    station_names_list=[]
    stations_list=[]
    dp=xr.open_dataset(dir+'station_numbers_95.nc')
    stations=dp.stations.values.tolist()
    lats=dp.lats.values.tolist()
    lons=dp.lons.values.tolist()
    elevs=dp.elevs.values.tolist()
    station_names=dp.station_names.values.tolist()
    points=zip(lons,lats)
    for i in range(len(stations)):
        print(str(i+1)+'/'+str(len(stations)))
        lat=lats[i]
        lon=lons[i]
        elev=elevs[i]
        station_name=station_names[i]
        station=stations[i]
        filename=stations[i]
        ds1=pd.read_csv(filename,header=None)
        nancount=0
        qcount=0
        yescount=0
        for row1 in ds1.iterrows():
            row_neg_space=row1[1][0].replace('-',' -')
            row_basic=row1[1][0]
            a=row_neg_space.split()
            code=a[0][-4:]
            if code=='PRCP':
                year=int(a[0][11:15])
                if yr_start<=year<=yr_end:
                    month=int(a[0][15:17])
                    if month in [1,3,5,7,8,10,12]:
                        days=31
                    if month in [4,6,9,11]:
                        days=30
                    if month==2 and year%4==0:
                        days=29
                    if month==2 and year%4!=0:
                        days=28

                    for i in range(days):
                        #print(row_basic)
                        mflag=row_basic[26+8*i]
                        qflag=row_basic[27+8*i]
                        sflag=row_basic[28+8*i]
                        obs=float(row_basic[21+8*i:26+8*i])
                    
                        if obs==-9999.0:
                            pass
                        elif qflag!=' ':
                            pass
                        else:
                            years.append(year)
                            months.append(month)
                            days1.append(i)
                            observations.append(obs/10.)#convert from tenths of mm to mm
                            lats1.append(lat)
                            lons1.append(lon)
                            elevs1.append(elev)
                            date=dt.datetime(year,month,i+1,6)
                            dates.append(date)
                            station_names_list.append(station_name)
                            stations_list.append(station)
    

In [ ]:
print(list(set(stations_list)))

In [ ]:
stations_short=list(set(stations_list))
if 1==1:
    date=pd.to_datetime(dt.datetime(2005,10,6,6))
    zipped=list(zip(lats1,lons1,dates,stations_list,observations))
    obs_mean_list=[]
    lats_new_list=[]
    lons_new_list=[]
    for i in range(len(stations_short)):
        station=stations_short[i]
        select_station=[x for x in zipped if x[3]==station and x[2]==date]
        select_obs=[x[4] for x in select_station]
        select_lat=[x[0] for x in select_station]
        select_lon=[x[1] for x in select_station]
        if len(select_obs)>0:
            obs_mean_list.append(select_obs[0])
            lats_new_list.append(select_lat[0])
            lons_new_list.append(select_lon[0])


if 1==1:#Now plot composite of extreme days
    clon=-70
    clat=35
    proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
    fig = plt.figure(figsize=(20,16))
    ax=plt.subplot(1,1,1,projection=proj_map)

    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
    ax.add_feature(cfeature.LAKES.with_scale('50m'))
    countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
    ax.add_feature(countries)
    ax.set_extent([-84,-66,36,48],crs=ccrs.PlateCarree())

    # *must* call draw in order to get the axis boundary used to add ticks:
    fig.canvas.draw()

    # Define gridline locations and draw the lines using cartopy's built-in gridliner:
    xticks = [-90,-85,-80,-75,-70,-65,-60,-50]
    yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
    ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0)
    ax.tick_params(labelsize=26)
    # Label the end-points of the gridlines using the custom tick makers:
    ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
    ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
    lambert_xticks(ax, xticks)
    lambert_yticks(ax, yticks)
    cax=ax.scatter(lons_new_list,lats_new_list,s=240,c=obs_mean_list,transform=ccrs.PlateCarree(),vmin=0,vmax=100,cmap=plt.cm.YlGnBu)
    cbar=plt.colorbar(cax,pad=0,extend='max',fraction=0.046)
    cbar.ax.tick_params(labelsize=26)
    cbar.set_label('mm/day',fontsize=24,rotation=90,labelpad=15)
    ax.set_title('Daily precipitation: '+month_dict[str(date.month)]+' '+str(date.day)+', '+str(date.year),fontsize=36)
    #props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
    #ax.text(0.01, 0.99,'n=314', transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props)
    plt.show()

In [ ]:
fig.savefig(dir+'neusa_example_synoptic_6.png')